## Preprocessing

In [74]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [75]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
cleaned_df= application_df.drop(columns= ['EIN', 'NAME'])

In [76]:
# display first 3 rows
cleaned_df.head(3)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [77]:
# Determine the number of unique values in each column.
columns_unique_values = cleaned_df.nunique(axis=0)
print(columns_unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [78]:
# Look at APPLICATION_TYPE value counts for binning
application_type_value_count = cleaned_df.APPLICATION_TYPE.value_counts()
application_type_value_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [79]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = application_type_value_count['T10']
application_types_to_replace = application_type_value_count[application_type_value_count < cutoff].index


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [80]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df['CLASSIFICATION'].value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [81]:
# You may find it helpful to look at CLASSIFICATION value counts >1
count_greater_than_one = class_count[class_count > 1]
count_greater_than_one

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [82]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = class_count['C2100']
classifications_to_replace = class_count[class_count < cutoff].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [83]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df)

In [84]:
# Split our preprocessed data into our features and target arrays
y = dummies_df['IS_SUCCESSFUL']
X = dummies_df.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [85]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [86]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_ft = len(X_train_scaled[0])
first_hidden_nodes = 6
second_hidden_nodes = 2

## Compile, Train and Evaluate the Model

In [87]:

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units= first_hidden_nodes, activation="relu", input_dim=input_ft))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=second_hidden_nodes, activation="relu", input_dim=input_ft))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 6)                 117678    
                                                                 
 dense_10 (Dense)            (None, 2)                 14        
                                                                 
 dense_11 (Dense)            (None, 1)                 3         
                                                                 
Total params: 117,695
Trainable params: 117,695
Non-trainable params: 0
_________________________________________________________________


In [88]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [89]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.9805 - accuracy: 0.6845
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5510 - accuracy: 0.7088
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.4850 - accuracy: 0.7364
Epoch 4/100
804/804 [==============================] - 4s 4ms/step - loss: 0.4263 - accuracy: 0.7850
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.3853 - accuracy: 0.8208
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3646 - accuracy: 0.8258
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3468 - accuracy: 0.8293
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3379 - accuracy: 0.8311
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.3364 - accuracy: 0.8311
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3358 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.1015 - accuracy: 0.9592
Epoch 82/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1008 - accuracy: 0.9594
Epoch 83/100
804/804 [==============================] - 3s 4ms/step - loss: 0.1008 - accuracy: 0.9595
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.1006 - accuracy: 0.9597
Epoch 85/100
804/804 [==============================] - 3s 4ms/step - loss: 0.1003 - accuracy: 0.9596
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.1004 - accuracy: 0.9597
Epoch 87/100
804/804 [==============================] - 3s 4ms/step - loss: 0.1001 - accuracy: 0.9593
Epoch 88/100
804/804 [==============================] - 3s 4ms/step - loss: 0.0999 - accuracy: 0.9595
Epoch 89/100
804/804 [==============================] - 4s 4ms/step - loss: 0.0997 - accuracy: 0.9596
Epoch 90/100
804/804 [==============================] - 3s 4ms/step - loss: 0.0996 - accuracy: 

In [90]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5328 - accuracy: 0.7928 - 505ms/epoch - 2ms/step
Loss: 0.5327926874160767, Accuracy: 0.7927696704864502


In [91]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')